# 先抓出news裡面對應是market 2天后大漲或大跌的那些文章

## 先把market 2天后大漲或大跌的那些日期抓出來

In [37]:
import csv 
from datetime import datetime
from datetime import timedelta

file_market = open('market_ready.csv', 'r')
reader_market = csv.DictReader(file_market)

time_delta = timedelta(days = -1)

up_list = []
up_intervel = []
down_list = []
down_intervel = []
for a_row in reader_market:
    two_day_return = float(a_row[u'2日報酬'])
    
    if two_day_return <= -0.0124:
        date = datetime.strptime( a_row['date'], "%Y-%m-%d")
        down_list.append( date.date() )
        down_intervel.append( (date + time_delta).date() )
        down_intervel.append( (date + time_delta + time_delta).date() )
    elif two_day_return >= 0.0135:
        date = datetime.strptime( a_row['date'], "%Y-%m-%d") 
        up_list.append(date.date())
        up_intervel.append( (date + time_delta).date() )
        up_intervel.append( (date + time_delta + time_delta).date() )
   
                 



In [38]:
len(up_list), len(down_list)

(74, 73)

## 有三天是有重疊的... 大跌之後跟著大漲 或著相反

In [22]:
set(up_intervel) & set(down_intervel)

{datetime.date(2016, 11, 7),
 datetime.date(2018, 3, 25),
 datetime.date(2018, 12, 3)}

In [24]:
time_delta = timedelta(days = 1) #可看出到底是哪一天是漲 哪一天是跌
intersect = set(up_intervel) & set(down_intervel)
for i in intersect:
    print( '{0} in up_list'.format(i+time_delta ), (i+time_delta) in up_list )
    print( '{0} in down_list'.format(i+time_delta ), (i+time_delta) in down_list )

2016-11-08 in up_list True
2016-11-08 in down_list False
2018-03-26 in up_list False
2018-03-26 in down_list True
2018-12-04 in up_list True
2018-12-04 in down_list False


## 所以我們知道 11/8是漲的樣本，11/9是跌的樣本，以此類推
## 下面我們可以得知，11/8是不需要留下的，只需要留下11/9的樣本即可

In [32]:
a_list = []
time_delta = timedelta(days = 1)
for i in intersect:
    a_list.append( i )
    a_list.append( i+time_delta )
    a_list.append( i+time_delta +time_delta)
a_list

[datetime.date(2016, 11, 7),
 datetime.date(2016, 11, 8),
 datetime.date(2016, 11, 9),
 datetime.date(2018, 3, 25),
 datetime.date(2018, 3, 26),
 datetime.date(2018, 3, 27),
 datetime.date(2018, 12, 3),
 datetime.date(2018, 12, 4),
 datetime.date(2018, 12, 5)]

In [33]:
import csv 
from datetime import datetime
from datetime import timedelta

file_market = open('market_ready.csv', 'r')
reader_market = csv.DictReader(file_market)



for a_row in reader_market:
    price = float(a_row['price'])
    date = datetime.strptime( a_row['date'], "%Y-%m-%d").date()
    one_day_ret = float(a_row[u'1日報酬'])
    two_day_ret = float(a_row[u'2日報酬'])
    if date in a_list:
        print(date, price, one_day_ret,two_day_ret)

2016-11-07 9189.84 0.013419495707503792 0.013517850466568185
2016-11-08 9217.43 0.0030022285480487305 0.016462012648666008
2016-11-09 8943.2 -0.029751243025441968 -0.026838334508544153
2018-03-26 10840.05 0.0015448110701604169 -0.015063820662484724
2018-03-27 10986.79 0.013536837929714494 0.015102560856963702
2018-12-03 10137.87 0.02526691363193681 0.025543834518924977
2018-12-04 10083.54 -0.005359113896706106 0.01977239146725892
2018-12-05 9916.74 -0.01654180972158598 -0.021812274175936463


## 只保留重疊的後一天

In [39]:
to_del = []
time_delta = timedelta(days = 1)
for i in intersect:
    to_del.append( i+time_delta )
    

file_market = open('market_ready.csv', 'r')
reader_market = csv.DictReader(file_market)

time_delta = timedelta(days = -1)

up_list = []
up_intervel = []
down_list = []
down_intervel = []
for a_row in reader_market:
    two_day_return = float(a_row[u'2日報酬'])
    date = datetime.strptime( a_row['date'], "%Y-%m-%d")
    if two_day_return <= -0.0124 and date.date() not in to_del:
        
        down_list.append( date.date() )
        down_intervel.append( (date + time_delta).date() )
        down_intervel.append( (date + time_delta + time_delta).date() )
    elif two_day_return >= 0.0135 and date.date() not in to_del:
        
        up_list.append(date.date())
        up_intervel.append( (date + time_delta).date() )
        up_intervel.append( (date + time_delta + time_delta).date() )
   
                 



In [138]:
len(up_list), len(down_list), set(up_intervel) & set(down_intervel)


(72, 72, set())

# 以下開始為新聞的EDA，初步先把news裡面的垃圾訊息排除，並留下有用新聞

In [70]:
df111 = pd.read_csv(r'C:\Users\User\Desktop\大數據與商業分析\mid tern\news.csv', encoding = 'big5')
df111

,id,p_type,s_name,s_area_name,post_time,title,author,content,page_url
0,1451580107540_N01,news,yahoo股市,重大要聞,2016/1/1 00:06:00,淪落的2015 竟是78年來最難獲利的一年,鉅亨網,鉅亨網編譯郭照青這一年，獲利真是難上難。<BR>根據CNBC分析報導，興業銀行的數據顯示，2...,https://tw.stock.yahoo.com/news_content/url/d/...
1,1451582807314_N01,news,yahoo股市,重大要聞,2016/1/1 00:52:00,美股盤中─今年最後交易日 道瓊大跌逾150點 注意油價走勢,鉅亨網,鉅亨網編譯郭照青今日為2015年最後交易日，美國股市中場走低，投資人注意油價走勢。<BR>道...,https://tw.stock.yahoo.com/news_content/url/d/...
2,1451584607515_N01,news,yahoo股市,重大要聞,2016/1/1 01:21:00,2016年 原油將突破每桶80美元,鉅亨網,鉅亨網編譯郭照青根據路透社分析師Andy Critchlow指出，原油價格下一波的大幅走勢將...,https://tw.stock.yahoo.com/news_content/url/d/...
3,1451597207172_N01,news,yahoo股市,重大要聞,2016/1/1 02:44:00,2015年 美國近70%投資人賠錢,鉅亨網,鉅亨網編譯郭照青拜拜了，2015年。<BR>根據CNNMoney分析報導，應用程式Openf...,https://tw.stock.yahoo.com/news_content/url/d/...
4,1451599007463_N01,news,yahoo股市,重大要聞,2016/1/1 03:30:00,【歐股盤後】全面收黑,中央社,（中央社台北2016年1月1日電）歐洲股市31日封關，三大股市全都收黑，倫敦股市提早收盤，金...,https://tw.stock.yahoo.com/news_content/url/d/...
5,1451599907245_N01,news,yahoo股市,重大要聞,2016/1/1 04:27:00,貴金屬盤後─黃金收高0.40美元 但今年大跌10.5%,鉅亨網,鉅亨網編譯郭照青2015年最後交易日，黃金期貨週四小幅上揚，但本週，本月與今年均下跌。<BR...,https://tw.stock.yahoo.com/news_content/url/d/...
6,1451600807160_N01,news,yahoo股市,重大要聞,2016/1/1 05:30:00,美股盤後─調整投資組合賣壓 道瓊收低178點 成交量清淡,鉅亨網,鉅亨網編譯郭照青週四，2015年最後交易日，儘管油價回穩，美國股市仍大幅收低，成交量清淡，<...,https://tw.stock.yahoo.com/news_content/url/d/...
7,1451603507368_N01,news,yahoo股市,重大要聞,2016/1/1 06:20:00,歐股全面收黑,法新社,（法新社倫敦31日電） 歐洲股市今天封關，三大股市全都收黑，倫敦股市提早收盤，金融時報百種股...,https://tw.stock.yahoo.com/news_content/url/d/...
8,1451603508604_N01,news,yahoo股市,重大要聞,2016/1/1 06:29:00,【美股盤後】封關收黑,中央社,（中央社台北2016年1月1日電）華爾街股市31日封關，揮別2015年，三大指數全都收黑，道...,https://tw.stock.yahoo.com/news_content/url/d/...
9,1451604407512_N01,news,yahoo股市,重大要聞,2016/1/1 06:57:00,【能源盤後】全年重挫三成,中央社,（中央社台北2016年1月1日電）世界油價31日上揚，年末雖跌幅縮減，但2015年仍以大跌作...,https://tw.stock.yahoo.com/news_content/url/d/...


In [71]:
df111.describe()

,id,p_type,s_name,s_area_name,post_time,title,author,content,page_url
count,247273,247273,247273,247273,247273,247273,247258,237798,247273
unique,247273,1,1,2,143138,236819,11,234135,247273
top,1531092438005_N01,news,yahoo股市,最新財經新聞,2018/6/28 08:29:00,工商時報晨訊,時報資訊,同上,https://tw.stock.yahoo.com/news_content/url/d/...
freq,1,247273,247273,139215,41,489,76713,2237,1


In [72]:
df112 = df111[df111['content']!=u'同上']

In [73]:
df112.describe()

,id,p_type,s_name,s_area_name,post_time,title,author,content,page_url
count,245036,245036,245036,245036,245036,245036,245021,235561,245036
unique,245036,1,1,2,142845,234617,11,234134,245036
top,1531092438005_N01,news,yahoo股市,最新財經新聞,2018/6/27 08:05:00,工商時報晨訊,時報資訊,待續,https://tw.stock.yahoo.com/news_content/url/d/...
freq,1,245036,245036,136978,41,489,76713,270,1


In [74]:
df113 = df112[df112['content']!=u'待續']
df113.describe()

,id,p_type,s_name,s_area_name,post_time,title,author,content,page_url
count,244766,244766,244766,244766,244766,244766,244751,235291,244766
unique,244766,1,1,2,142845,234614,11,234133,244766
top,1531092438005_N01,news,yahoo股市,最新財經新聞,2018/6/27 08:05:00,工商時報晨訊,時報資訊,本日資訊傳送完畢,https://tw.stock.yahoo.com/news_content/url/d/...
freq,1,244766,244766,136711,41,489,76713,101,1


In [75]:
df114 = df113[df113['content']!=u'本日資訊傳送完畢']
df114.describe()

,id,p_type,s_name,s_area_name,post_time,title,author,content,page_url
count,244665,244665,244665,244665,244665,244665,244650,235190,244665
unique,244665,1,1,2,142812,234613,11,234132,244665
top,1531092438005_N01,news,yahoo股市,最新財經新聞,2018/6/28 08:29:00,工商時報晨訊,時報資訊,如何在Yahoo股市APP看直播？<BR> ★IOS用戶：請點選右上角的「新聞」，再點擊上方...,https://tw.stock.yahoo.com/news_content/url/d/...
freq,1,244665,244665,136610,41,489,76713,22,1


In [95]:
mask = df114['content'].str.contains(u'如何在Yahoo股市APP看直播', regex=False, na = False)
df115 = df114[~mask]
df115.describe()

,id,p_type,s_name,s_area_name,post_time,title,author,content,page_url
count,244630,244630,244630,244630,244630,244630,244615,235155,244630
unique,244630,1,1,2,142780,234578,11,234127,244630
top,1531092438005_N01,news,yahoo股市,最新財經新聞,2018/6/28 08:29:00,工商時報晨訊,時報資訊,近期上市上櫃公司實施庫藏股一覽表：<BR>代號 公司名 買回張數 區間價格(元) 買回期間<...,https://tw.stock.yahoo.com/news_content/url/d/...
freq,1,244630,244630,136610,41,489,76713,7,1


In [96]:
mask = df115['content'].str.contains(u'獎號碼', regex=False, na = False)
df116 = df115[~mask]

In [97]:
mask = df116['content'].str.contains(u'威力彩', regex=False, na = False)
df117 = df116[~mask]

In [99]:
mask = df117['content'].str.contains(u'彩券', regex=False, na = False)
df118 = df117[~mask]

In [106]:
mask = df118['title'].str.contains(u'統一發票', regex=False, na = False)
df119 = df118[~mask]

In [110]:
mask = df119['title'].str.contains(u'頭獎', regex=False, na = False)
df120 = df119[~mask]

In [112]:
mask = df120['title'].str.contains(u'行事曆', regex=False, na = False)
df121 = df120[~mask]
df120[mask]

,id,p_type,s_name,s_area_name,post_time,title,author,content,page_url
614,1452211676184_N01,news,yahoo股市,重大要聞,2016/1/8 08:00:00,2016/1/8財經行事曆,Yahoo奇摩股市,2016/1/8財經行事曆<BR>今日國內外財經焦點：<BR>台灣：<BR>1.新股掛牌：F...,https://tw.stock.yahoo.com/news_content/url/d/...
2322,1453768129515_N01,news,yahoo股市,重大要聞,2016/1/26 08:15:00,1/26財經行事曆,Yahoo奇摩股市,今日國內外財經焦點：<BR>台灣：<BR>1.新股掛牌：宏觀(6568)<BR>2.股東會：...,https://tw.stock.yahoo.com/news_content/url/d/...
2410,1453805918029_N01,news,yahoo股市,重大要聞,2016/1/26 18:45:00,2016/1/27財經行事曆,Yahoo奇摩股市,今日國內外財經焦點：<BR>台灣：<BR>1.新股掛牌：F-英利(2239)、F-紅馬(29...,https://tw.stock.yahoo.com/news_content/url/d/...
2568,1453892318745_N01,news,yahoo股市,重大要聞,2016/1/27 18:40:00,2016/1/28財經行事曆,Yahoo奇摩股市,今日國內外財經焦點：<BR>台灣：<BR>1.股東會：矽品(2325)、佳晶科(5242)、...,https://tw.stock.yahoo.com/news_content/url/d/...
2711,1453978130035_N01,news,yahoo股市,重大要聞,2016/1/28 18:31:00,2016/1/29財經行事曆,Yahoo奇摩股市,今日國內外財經焦點：<BR>台灣：<BR>1.新股掛牌：心悅(6575)<BR>2.股東會：...,https://tw.stock.yahoo.com/news_content/url/d/...
3023,1454228329825_N01,news,yahoo股市,重大要聞,2016/1/31 16:10:00,2016/2/1財經行事曆,Yahoo奇摩股市,今日國內外財經焦點：<BR>台灣：<BR>1.法說會：聯發科(2454)<BR>中國：<BR...,https://tw.stock.yahoo.com/news_content/url/d/...
3111,1454321034477_N01,news,yahoo股市,重大要聞,2016/2/1 17:47:00,2016/2/2財經行事曆,Yahoo奇摩股市,今日國內外財經焦點：<BR>台灣：<BR>1.法說會：超豐(2441)、聯詠(3034)、文...,https://tw.stock.yahoo.com/news_content/url/d/...
3244,1454409228261_N01,news,yahoo股市,重大要聞,2016/2/2 18:16:00,2016/2/3財經行事曆,Yahoo奇摩股市,今日國內外財經焦點：<BR>台灣：<BR>1.股東會：千興(2025)<BR>2.法說會：宏...,https://tw.stock.yahoo.com/news_content/url/d/...
3376,1454504173153_N01,news,yahoo股市,重大要聞,2016/2/3 20:39:00,2016/2/4財經行事曆,Yahoo奇摩股市,今日國內外財經焦點：<BR>台灣：<BR>1.台股今日休市<BR>2.股東會：聯亞藥(656...,https://tw.stock.yahoo.com/news_content/url/d/...
3448,1454581573947_N01,news,yahoo股市,重大要聞,2016/2/4 18:15:00,2016/2/5財經行事曆,Yahoo奇摩股市,今日國內外財經焦點：<BR>台灣：<BR>1.台股今日休市<BR>2.股東會：華廣(4737...,https://tw.stock.yahoo.com/news_content/url/d/...


In [118]:
df_len = df121['content'].fillna('').apply(len)
df_len.describe()

count    243041.000000
mean        607.546513
std         498.083274
min           0.000000
25%         271.000000
50%         526.000000
75%         817.000000
max       16338.000000
Name: content, dtype: float64

In [123]:
mask = df121[df_len <=30 ]
mask

,id,p_type,s_name,s_area_name,post_time,title,author,content,page_url,length_content
107,1451867212458_N01,news,yahoo股市,重大要聞,2016/1/4 07:58:00,廢證所稅 催化資金動能(工商時報，無內文),Moneydj理財網,NaN,https://tw.stock.yahoo.com/news_content/url/d/...,243041
108,1451867213978_N01,news,yahoo股市,重大要聞,2016/1/4 07:58:00,退撫+勞動新銀彈 3240億元入台股(工商時報，無內文),Moneydj理財網,NaN,https://tw.stock.yahoo.com/news_content/url/d/...,243041
109,1451867215197_N01,news,yahoo股市,重大要聞,2016/1/4 07:59:00,住宅去年使照核發戶數 7年新高(工商時報，無內文),Moneydj理財網,NaN,https://tw.stock.yahoo.com/news_content/url/d/...,243041
110,1451867216394_N01,news,yahoo股市,重大要聞,2016/1/4 07:59:00,淡季效應 蘋果供應鏈盤整(工商時報，無內文),Moneydj理財網,NaN,https://tw.stock.yahoo.com/news_content/url/d/...,243041
111,1451867217656_N01,news,yahoo股市,重大要聞,2016/1/4 08:00:00,售屋所得課稅率 南增北降(經濟日報，無內文),Moneydj理財網,NaN,https://tw.stock.yahoo.com/news_content/url/d/...,243041
480,1452123132697_N01,news,yahoo股市,重大要聞,2016/1/7 07:09:00,今日記事,財訊快報,1.大中(6435)今日掛牌上櫃交易。,https://tw.stock.yahoo.com/news_content/url/d/...,243041
488,1452126735129_N01,news,yahoo股市,重大要聞,2016/1/7 08:02:00,戴德梁行：今年房市 8年寒冬開始(工商時報，無內文),Moneydj理財網,NaN,https://tw.stock.yahoo.com/news_content/url/d/...,243041
489,1452126736971_N01,news,yahoo股市,重大要聞,2016/1/7 08:03:00,北韓氫彈試爆 東北亞緊張(經濟日報，無內文),Moneydj理財網,NaN,https://tw.stock.yahoo.com/news_content/url/d/...,243041
616,1452212568928_N01,news,yahoo股市,重大要聞,2016/1/8 08:03:00,出口恐連11黑 七年最遭(經濟日報，無內文),Moneydj理財網,NaN,https://tw.stock.yahoo.com/news_content/url/d/...,243041
617,1452212570150_N01,news,yahoo股市,重大要聞,2016/1/8 08:03:00,財部救台股 急call公股護盤(經濟日報，無內文),Moneydj理財網,NaN,https://tw.stock.yahoo.com/news_content/url/d/...,243041


In [130]:
df122 = df121[ df_len > 30 ].drop('length_content', axis = 1)
df122

,id,p_type,s_name,s_area_name,post_time,title,author,content,page_url
0,1451580107540_N01,news,yahoo股市,重大要聞,2016/1/1 00:06:00,淪落的2015 竟是78年來最難獲利的一年,鉅亨網,鉅亨網編譯郭照青這一年，獲利真是難上難。<BR>根據CNBC分析報導，興業銀行的數據顯示，2...,https://tw.stock.yahoo.com/news_content/url/d/...
1,1451582807314_N01,news,yahoo股市,重大要聞,2016/1/1 00:52:00,美股盤中─今年最後交易日 道瓊大跌逾150點 注意油價走勢,鉅亨網,鉅亨網編譯郭照青今日為2015年最後交易日，美國股市中場走低，投資人注意油價走勢。<BR>道...,https://tw.stock.yahoo.com/news_content/url/d/...
2,1451584607515_N01,news,yahoo股市,重大要聞,2016/1/1 01:21:00,2016年 原油將突破每桶80美元,鉅亨網,鉅亨網編譯郭照青根據路透社分析師Andy Critchlow指出，原油價格下一波的大幅走勢將...,https://tw.stock.yahoo.com/news_content/url/d/...
3,1451597207172_N01,news,yahoo股市,重大要聞,2016/1/1 02:44:00,2015年 美國近70%投資人賠錢,鉅亨網,鉅亨網編譯郭照青拜拜了，2015年。<BR>根據CNNMoney分析報導，應用程式Openf...,https://tw.stock.yahoo.com/news_content/url/d/...
4,1451599007463_N01,news,yahoo股市,重大要聞,2016/1/1 03:30:00,【歐股盤後】全面收黑,中央社,（中央社台北2016年1月1日電）歐洲股市31日封關，三大股市全都收黑，倫敦股市提早收盤，金...,https://tw.stock.yahoo.com/news_content/url/d/...
5,1451599907245_N01,news,yahoo股市,重大要聞,2016/1/1 04:27:00,貴金屬盤後─黃金收高0.40美元 但今年大跌10.5%,鉅亨網,鉅亨網編譯郭照青2015年最後交易日，黃金期貨週四小幅上揚，但本週，本月與今年均下跌。<BR...,https://tw.stock.yahoo.com/news_content/url/d/...
6,1451600807160_N01,news,yahoo股市,重大要聞,2016/1/1 05:30:00,美股盤後─調整投資組合賣壓 道瓊收低178點 成交量清淡,鉅亨網,鉅亨網編譯郭照青週四，2015年最後交易日，儘管油價回穩，美國股市仍大幅收低，成交量清淡，<...,https://tw.stock.yahoo.com/news_content/url/d/...
7,1451603507368_N01,news,yahoo股市,重大要聞,2016/1/1 06:20:00,歐股全面收黑,法新社,（法新社倫敦31日電） 歐洲股市今天封關，三大股市全都收黑，倫敦股市提早收盤，金融時報百種股...,https://tw.stock.yahoo.com/news_content/url/d/...
8,1451603508604_N01,news,yahoo股市,重大要聞,2016/1/1 06:29:00,【美股盤後】封關收黑,中央社,（中央社台北2016年1月1日電）華爾街股市31日封關，揮別2015年，三大指數全都收黑，道...,https://tw.stock.yahoo.com/news_content/url/d/...
9,1451604407512_N01,news,yahoo股市,重大要聞,2016/1/1 06:57:00,【能源盤後】全年重挫三成,中央社,（中央社台北2016年1月1日電）世界油價31日上揚，年末雖跌幅縮減，但2015年仍以大跌作...,https://tw.stock.yahoo.com/news_content/url/d/...


In [131]:
df_final = df122.drop_duplicates(subset ='content')
df_final.describe()

,id,p_type,s_name,s_area_name,post_time,title,author,content,page_url
count,232514,232514,232514,232514,232514,232514,232499,232514,232514
unique,232514,1,1,2,135037,222873,11,232514,232514
top,1531092438005_N01,news,yahoo股市,最新財經新聞,2018/6/28 08:29:00,工商時報晨訊,時報資訊,（中央社記者鄭崇生華盛頓2017年4月2日專電）美國總統川普今天說，他非常尊重中國國家主席習...,https://tw.stock.yahoo.com/news_content/url/d/...
freq,1,232514,232514,135083,40,488,68079,1,1


In [132]:
df_final.to_csv('news_ready.csv', encoding = 'big5')

## 經過以上步驟，我將重複的、關於彩票及發票的、關於行事曆、以及內容過少的新聞排除

# 接著讀取文章，並分類為上漲及下跌文章群

In [141]:
import csv 
from datetime import datetime
from datetime import timedelta

news_content_plus_title = dict()
news_content_plus_title['up'] = [] 
news_content_plus_title['down'] = [] 

time_delta = timedelta(hours = -13.5)

with open( 'news_ready.csv', encoding = 'big5') as f:

    csv_reader = csv.DictReader(f)
    for a_row in csv_reader:
        date = datetime.strptime( a_row['post_time'], "%Y/%m/%d %H:%M:%S") 
        adj_date = (date+time_delta).date()
        title = a_row['title']
        content = a_row['content']
        if adj_date in up_intervel:
            news_content_plus_title['up'].append( title +'＜' +content   )
        elif adj_date in down_intervel:
            news_content_plus_title['down'].append( title +'＜' +content   )



In [142]:
import pandas as pd
df = pd.DataFrame.from_dict(news_content_plus_title, 'index')
df

,0,1,2,3,4,5,6,7,8,9,...,33537,33538,33539,33540,33541,33542,33543,33544,33545,33546
up,《政治》獲頒當選證書，蔡英文：人民給的＜中選會昨頒贈當選證書給正副總統當選人蔡英文及陳建仁。...,BBC妙聞 iPhone用谷歌搜尋 蘋果收300億＜（中央社倫敦2016年1月22日綜合外電...,《政治》在美受訓墜毀，我F-16飛官罹難＜中華民國空軍飛行員高鼎程赴美路克基地受訓，台灣時間...,社福津貼調高囉 每月增加73到600元＜（中央社記者陳政偉台北2016年1月23日電）國家發...,技嘉葉培城：今年網通、電競產品帶頭 獲利拚回2014年水位＜鉅亨網記者蔡宗憲 台北主機板大廠...,技嘉布局新領域產品 從手持裝置往外擴散＜鉅亨網記者蔡宗憲 台北主機板廠技嘉 (2376) 今...,主辦奧運日近 巴西經濟危機罩頂＜（法新社巴西聖保羅23日電） 巴西期待2016年有好的開始，...,迎接奧運年 巴西經濟危機罩頂＜（法新社巴西聖保羅23日電） 巴西期待2016年有好的開始，今...,世界主要報紙頭條新聞標題之二＜（中央社台北2016年1月23日綜合報導）以下是世界各地主要報...,法經長質疑 大陸官方誇大成長數字＜（法新社達佛斯22日電） 法國經濟部長馬卡龍（Emmanu...,...,None,None,None,None,None,None,None,None,None,None
down,《國際金融》全球股市最慘開年，首周跌逾2.3兆美元＜受中國經濟亂流攪局，全球股市開年以來出師...,《政治》民進黨宣告：未來對外經貿談判將採黑箱作業＜民進黨表示，未來兩岸及國際經貿談判應仿效美...,《電腦設備》英業達法人積極買進＜NB代工廠英業達 （2356） 上周股價相對持穩，單周僅下跌...,花花公子創辦人海夫納傳將出售豪宅＜（法新社洛杉磯10日電） 美國媒體報導，「花花公子」（Pl...,美元兌日圓做多者一夜白了頭、週線創兩年最大貶幅＜MoneyDJ新聞 2016-01-11 0...,《美股》道瓊五日崩逾千點、史上最慘開春！VIX飆＜MoneyDJ新聞 2016-01-11 ...,《美債》就業人數增、薪情卻欠佳！殖利率逾10週低＜MoneyDJ新聞 2016-01-11 ...,日股今日因國定假日「成人日」休市一天＜MoneyDJ新聞 2016-01-11 07:08:...,晨間解析：蘋概股反攻，台股收復10年線，選前法人觀望，周K下挫444點＜中國股匯重挫、油價大...,今日記事＜1.嘉晶(3016)合併漢磊增資新股權利證書上市。<BR>2.夠麻吉(8472)今...,...,《社會》蔡正元中影背信案，獲判無罪定讞＜前立委蔡正元被控在12年前擔任中影董事長時，指示以增...,怕明年太慘！陸企拚死赴海外IPO、募款額腰斬在所不惜＜MoneyDJ新聞 2018-12-2...,【台股盤中】電子權值股穩盤 量能恐再創低＜（中央社記者田裕斌台北2018年12月26日電）在...,機構：陸明年定向降準與TMLF仍可期，全面降息機率低＜MoneyDJ新聞 2018-12-2...,台股小漲營建股走紅 華固近日波動大＜（中央社記者韋樞台北2018年12月26日電）台股25日...,《傳產》賴正鎰：今年房市推案量「跳空漲停」，明年估再增5%＜全國商總理事長、鄉林集團董事長賴...,台股盤中—量縮反彈無力 被動、5G概念領軍 力守9500點＜台股今 (26) 日跌深反彈，在...,台股今恐改寫正常交易日新低量 封關當周成交量恐不到2000億元＜台股封關倒數，封關前行情隨著...,陸市場准入負面清單全國性實施，續列管金融/互聯網＜MoneyDJ新聞 2018-12-26 ...,哈森泰博：Fed持續緊縮、10年債殖利率明年底上看4%＜MoneyDJ新聞 2018-12-...


In [145]:
df1 = df.T
print(df1.shape)
print(df1['up'].describe())
df1['down'].describe()

(33547, 2)
count                                                 28526
unique                                                28526
top       《熱門族群》學生488吃到飽、3G升4G，電信3雄無限「延」伸＜繼中華電 (2412) 丟出...
freq                                                      1
Name: up, dtype: object


count                                                 33547
unique                                                33547
top       永豐金證券獲中華信評調升評等 成績歷來最佳＜永豐金證券轉型發展特色財富管理業務、並強化風險管...
freq                                                      1
Name: down, dtype: object

In [146]:
df1.to_csv('news_UandD.csv', encoding = 'big5')